In [2]:
from ultralytics import YOLO
import cv2
import numpy as np
from PIL import Image

def visualize_output(image, boxes, keypoints):
    """
    Visualize the output model on the input image.

    Parameters:
    image (numpy.ndarray): The original image.
    boxes (numpy.ndarray): Array of bounding boxes, each box is [x, y, width, height].
    keypoints (numpy.ndarray): Array of keypoints for pose estimation.
    """

    result_image = image.copy()
    
    # Font setting for the numbers
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_thickness = 2
    
    # Draw bounding boxes
    for i, box in enumerate(boxes):
        center_x, center_y, width, height = box
        top_left_x = int(center_x - width / 2)
        top_left_y = int(center_y - height / 2)
        bottom_right_x = int(top_left_x + width)
        bottom_right_y = int(top_left_y + height)

        # Draw the box
        #cv2.rectangle(result_image, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), (0, 255, 0), 2)
        
        # # Write the number of the box
        # text = str(i)
        # (text_width, text_height), _ = cv2.getTextSize(text, font, font_scale, font_thickness)
        # text_offset_x = top_left_x + text_width
        # text_offset_y = top_left_y + text_height
        # box_coords = ((text_offset_x, text_offset_y - text_height), (top_left_x, text_offset_y + 4))
        # cv2.rectangle(result_image, box_coords[0], box_coords[1], (255, 255, 255), cv2.FILLED)
        # cv2.putText(result_image, text, (top_left_x, text_offset_y), font, font_scale, (0, 0, 255), font_thickness)

    # Draw keypoints
    for keypoint in keypoints:
        for x, y in keypoint:
            cv2.circle(result_image, (int(x), int(y)), 20, (255, 0, 0), -1)
    
    return result_image

def plot_results(image, mode="inline", scale=1):
    """
    Displays an image using either a popup window or inline display.

    Parameters
    ----------
    image : numpy.ndarray or CV2 image
        The image to display.
    mode : str, optional
        The display mode. Must be either "popup" or "inline". If "popup", the image is displayed in a popup window. If "inline", the image is displayed inline in the notebook. Default is "inline".
    scale : float, optional
        The scaling factor for the image. Default is 1.

    Raises
    ------
    ValueError
        If the mode is not "popup" or "inline".
    """
    
    if mode == "popup":
        pass
    elif mode == "inline":
        if type(image) == np.ndarray:
            # Convert from BGR to RGB (because OpenCV uses BGR order for color channels, whereas PIL uses RGB.)
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = Image.fromarray(image_rgb)
    else:
        raise ValueError("Mode must be either 'popup' or 'inline'")

    if type(image) is np.ndarray:
        cv2.imshow('Image', image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        width, height = image.size
        display(image.resize((int(width*scale), int(height*scale))))

### load the model

In [3]:
model = YOLO('yolov8n-pose.pt')

#### small example image prediction with visualization

In [7]:
# Predict with the model
image = cv2.imread('example_images/yellow3.jpg') 

results = model(image)

boxes = results[0].boxes.xywh  # Boxes object for bbox outputs
keypoints = results[0].keypoints.xy  # Keypoints object for pose outputs



0: 640x480 1 person, 78.1ms
Speed: 2.9ms preprocess, 78.1ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 480)


the keypoints are also calculated in 3D which we will leverage for our predictions

In [22]:
#print(results)
keypoints_all = results[0].keypoints
#print(keypoints_3d)
print(keypoints_all.data)
print(keypoints_all.data.shape)

tensor([[[1.2903e+03, 1.4783e+03, 9.9350e-01],
         [1.3883e+03, 1.3790e+03, 9.9267e-01],
         [1.2600e+03, 1.3812e+03, 7.7818e-01],
         [1.7325e+03, 1.3233e+03, 9.6801e-01],
         [0.0000e+00, 0.0000e+00, 4.2132e-02],
         [1.9563e+03, 1.7039e+03, 9.9153e-01],
         [1.6893e+03, 1.5877e+03, 9.9206e-01],
         [1.5165e+03, 2.2164e+03, 9.7732e-01],
         [1.2452e+03, 1.8650e+03, 9.7607e-01],
         [1.3291e+03, 2.2291e+03, 9.8469e-01],
         [8.3613e+02, 2.0155e+03, 9.8376e-01],
         [2.4183e+03, 2.8356e+03, 9.8898e-01],
         [2.2288e+03, 2.7412e+03, 9.9286e-01],
         [1.5074e+03, 2.8463e+03, 9.8288e-01],
         [1.4614e+03, 2.7262e+03, 9.8562e-01],
         [1.2421e+03, 3.3962e+03, 9.0765e-01],
         [1.3356e+03, 3.2737e+03, 9.3313e-01]]])
torch.Size([1, 17, 3])


In [8]:
result = visualize_output(image, boxes, keypoints)
# plot_results(result)

In [39]:
# Save the image for the mockup:
cv2.imwrite('ari_yellow3_kps.jpg', result)

True

## fine tuning yolo for our classification task

- option 1: freeze the complete model, use it as a feature extractor and add a classification head to be trained on our dataset
- option 2: cut the keypoint output layer, freeze the rest of the model and add a classification head to be trained on our dataset

#### 1) using yolo as feature extractor:

freezing the yolo model:

In [ ]:
## freezing the model by setting the requires_grad attribute to False:
for param in model.parameters():
    param.requires_grad = False

defining a classification layer:

In [ ]:
import torch
import torch.nn as nn

# `feature_extractor` is the part of the model suitable for feature extraction

class YOLO_kp_Classifier(nn.Module):
    def __init__(self, num_keypoints, num_classes=3):
        super(YOLO_kp_Classifier, self).__init__()
        # to flatten the output 
        self.flatten = nn.Flatten()
        # add new classification layer(s) to the model
        self.classifier = nn.Sequential(
            nn.Linear(num_keypoints* 3, 512),  # 17 keypoints * 3 (x, y, z coordinates for each keypoint)
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes),
        )
    
    def forward(self, keypoints_3d):
        keypoints_3d = self.flatten(keypoints_3d)
        output = self.classifier(keypoints_3d)
        return output


initializing the model:

In [ ]:
# initialize the classifier model
num_classes = 3  #  3-class classification problem
num_keypoints = 17  # 17 keypoints in the model

kp_classifier_model = YOLO_kp_Classifier(num_keypoints=num_keypoints, num_classes=num_classes)

load the own dataset:

In [ ]:
from torch.utils.data import random_split, DataLoader, Subset
from torchvision import datasets, transforms

In [ ]:
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)), # Resize images for the model?
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # normalize
])

In [ ]:
own_dataset = datasets.ImageFolder(root='path/to/your/data', transform=data_transforms)

splitting the data into train, validation and test set:

In [ ]:
generator1 = torch.Generator().manual_seed(13)  # set seed for reproducibility of the split
train_and_val_dataset, test_dataset =random_split(own_dataset, [0.8, 0.2], generator=generator1)  # 80% training and evaluation, 20% testing

#train_size = int(0.6 * len(own_dataset))
#val_size = int(0.15 * len(own_dataset))
#test_size = len(own_dataset) - train_size - val_size
#train_dataset, val_dataset, test_dataset = random_split(own_dataset , [train_size, val_size, test_size], generator=generator1)

dataloader: (not needed anymore here?)

In [ ]:
def setup_data_loaders(batch_size, train_dataset, val_dataset):
    # Assuming `train_dataset` and `val_dataset` are already defined
    training_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    validation_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
    return training_loader, validation_loader

feature extraction with yolo:

In [ ]:
# Placeholder for obtaining keypoints from the YOLO model
def get_keypoints_from_yolo(model, inputs):
    model.eval()  # YOLO model set to evaluation mode because we are not training it
    with torch.no_grad(): # gradients are not computed for the frozen model
        results = model(inputs)
        keypoints = results[0].keypoints.data  # extract the keypoints from the results
    return keypoints


In [ ]:
def process_keypoints_for_classifier(keypoints):
    # processing the keypoints in a suitable format for the classifier
    processed_data = ...  # normalization, flattening, etc.
    return processed_data

#### training the model:

- with cross-entropy loss function (fits our classification task)
- Adam optimizer

HPO:

In [25]:
#!pip install --upgrade pip
#%pip install scikit-learn
#%pip install optuna
import optuna
from sklearn.metrics import f1_score, confusion_matrix
import numpy as np

In [ ]:
# initialize the classifier model
num_classes = 3  #  3-class classification problem
num_keypoints = 17  # 17 keypoints in the model
# loss function and optimizer
criterion = nn.CrossEntropyLoss()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu"))
print(f"Using device: {device}")

In [ ]:
def train_and_eval_model(model, optimizer, num_epochs, lr, momentum, batch_size, trial, train_loader, val_loader, device):
    
    kp_classifier_model = kp_classifier_model.to(device)  # move model to device
    kp_classifier_model.train()  # set model to training mode

    for epoch in range(num_epochs):
        for inputs in train_loader:  # Assuming data_loader is your DataLoader instance for the dataset
            inputs, labels = inputs.to(device), labels.to(device)  # move data to device
            model = model.to(device) # YOLO move model to device
            optimizer.zero_grad()  # Zero the parameter gradients
            keypoints = get_keypoints_from_yolo(model, inputs) # get keypoints from the YOLO model
            processed_kps = process_keypoints_for_classifier(keypoints) # prepare the keypoints for the classifier
            classification_output = kp_classifier_model(processed_kps) # get results for the classification 
            loss = criterion(classification_output , labels)  # Compute loss
            loss.backward()  # Backpropagate the loss
            optimizer.step()  # Update weights

            # todo: further processing, such as calculating accuracy or loss, goes here

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
            
        
        kp_classifier_model.eval()  # model to evaluation mode

        total_loss = 0
        all_predictions = []
        all_labels = []

        with torch.no_grad():  # no need to compute gradients, because we are in evaluation mode
            for inputs, labels in val_loader:  # iterate over validation dataset
                inputs, labels = inputs.to(device), labels.to(device)  # move data to device
                keypoints = get_keypoints_from_yolo(model, inputs) # get keypoints from the YOLO model
                processed_kps = process_keypoints_for_classifier(keypoints) # prepare the keypoints for the classifier
                classification_output = kp_classifier_model(processed_kps) # get results for the classification 
                loss = criterion(classification_output , labels)  # compute loss
                total_loss += loss.item()  # accumulate the loss
                # get predictions for output
                _, predicted = torch.max(classification_output.data, 1)
                # collect the predictions and labels
                all_predictions.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
            # calculate the validation metrics:
            avg_loss = total_loss / len(val_loader)  # get the average loss
            accuracy = (np.array(all_predictions) == np.array(all_labels)).mean()
            f1 = f1_score(all_labels, all_predictions, average='weighted')  
            conf_matrix = confusion_matrix(all_labels, all_predictions)
            
            print(f"Validation Loss: {avg_loss:.4f}")
            print(f"Validation Accuracy: {accuracy:.4f}")
            print(f"Validation F1 Score: {f1:.4f}")
            print("Confusion Matrix:")
            print(conf_matrix)
            
            # here: F1 score chosen as the metric to optimize
            # other options: combining metrics like accuracy and F1 score to maximize on both
            #                        or multi-objective optimization on F1 score and accuracy
    return f1 

In [ ]:
# this function was generated by gpt-4

def get_k_fold_indices(n, k=5, random_seed=None):
    """
    Generate indices for k-fold cross-validation.

    Parameters:
    - n: Total number of samples in the dataset.
    - k: Number of folds.
    - random_seed: Optional seed for reproducibility.

    Returns:
    - A list of tuples, each containing (train_indices, val_indices) for a fold.
    """
    # Initialize the random generator
    g = torch.Generator()
    if random_seed is not None:
        g.manual_seed(random_seed)
    
    # Generate a random permutation of indices
    indices = torch.randperm(n, generator=g).tolist()
    
    # Calculate fold sizes
    fold_sizes = [n // k for _ in range(k)]
    for i in range(n % k):
        fold_sizes[i] += 1
    
    # Generate train and validation indices for each fold
    current = 0
    folds_indices = []
    for fold_size in fold_sizes:
        start, end = current, current + fold_size
        val_indices = indices[start:end]
        train_indices = indices[:start] + indices[end:]
        folds_indices.append((train_indices, val_indices))
        current = end
    
    return folds_indices


In [ ]:
def objective(trial):
    # Define hyperparameters to optimize
    lr = trial.suggest_categorical("lr", [1e-4, 5e-4, 1e-3])
    momentum = trial.suggest_categorical("momentum", [0.9, 0.95])
    batch_size = trial.suggest_categorical("batch_size", [4, 8, 16])
    num_epochs = trial.suggest_categorical("num_epochs", [100, 200, 300])

    # Set up data loaders with the chosen batch size
    # Note: You might need to modify your dataset preparation code to accommodate dynamic batch sizes
    #train_loader, val_loader = setup_data_loaders(batch_size)
    # Your dataset
    # Convert dataset into a list to enable index-based access
    dataset = list(train_and_val_dataset)

    validation_scores = []

    n = len(dataset)
    folds_indices = get_k_fold_indices(n, k, random_seed=13)
    k = 5
    for fold, (train_idx, val_idx) in enumerate(folds_indices, start=1):
        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)
        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

        # Here, implement the training and validation for each fold
        # This includes model initialization, training, and evaluation
        # Initialize model and optimizer with the chosen hyperparameters
        kp_classifier_model = YOLO_kp_Classifier(num_keypoints=num_keypoints, num_classes=num_classes)
        optimizer = torch.optim.Adam(kp_classifier_model.classifier.parameters(), lr=lr, momentum=momentum, weight_decay=1e-5)
    
        # Train and evaluate the model
        validation_score = train_and_eval_model(model, optimizer, num_epochs, lr, momentum, batch_size, trial, train_loader, val_loader, device)
        validation_scores.append(validation_score)

    return np.mean(validation_scores)




In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)  # run 20 trials

# Best hyperparameters
print("Best hyperparameters:", study.best_params)


In [ ]:
best_params = study.best_params
# Re-setup data loaders, model, and optimizer using best_params...
# Train your final model...


In [ ]:
# Hyperparameters ---> todo: HPO
num_epochs = 50 # check for appropriate number of epochs
learning_rate = 0.001  # check for appropriate learning rate

train:

In [ ]:
# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(kp_classifier_model.classifier.parameters(), lr=learning_rate)


kp_classifier_model.train()  # set model to training mode

for epoch in range(num_epochs):
    for inputs in train_loader:  # Assuming data_loader is your DataLoader instance for the dataset
        optimizer.zero_grad()  # Zero the parameter gradients
        keypoints = get_keypoints_from_yolo(model, inputs) # get keypoints from the YOLO model
        processed_kps = process_keypoints_for_classifier(keypoints) # prepare the keypoints for the classifier
        classification_output = kp_classifier_model(processed_kps) # get results for the classification 
        loss = criterion(classification_output , labels)  # Compute loss
        loss.backward()  # Backpropagate the loss
        optimizer.step()  # Update weights

        # todo: further processing, such as calculating accuracy or loss, goes here

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
        
    
    kp_classifier_model.eval()  # model to evaluation mode

    total_loss = 0
    with torch.no_grad():  # no need to compute gradients, because we are in evaluation mode
        for inputs, labels in val_loader:  # iterate over validation dataset
            keypoints = get_keypoints_from_yolo(model, inputs) # get keypoints from the YOLO model
            processed_kps = process_keypoints_for_classifier(keypoints) # prepare the keypoints for the classifier
            classification_output = kp_classifier_model(processed_kps) # get results for the classification 
            loss = criterion(classification_output , labels)  # compute loss
            total_loss += loss.item()  # accumulate the loss
        avg_loss = total_loss / len(val_loader)  # get the average loss
        print(f"Validation Loss: {avg_loss}")




evaluate the model:

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
kp_classifier_model.eval()  # model is in evaluation mode

with torch.no_grad():  # no gradient computation, because of evaluation mode
    for inputs, labels in test_loader:  # iterate over test dataset
        keypoints = get_keypoints_from_yolo(model, inputs)
        prepared_data = process_keypoints_for_classifier(keypoints)
        outputs = kp_classifier_model(prepared_data)

        # todo: perform evaluation e.g., compute accuracy, confusion matrix, f1-score, etc.

